# Download New Yorker cartoons

We'll be scraping images and captions from their [Random cartoon page](https://www.newyorker.com/cartoons/random/).

In [19]:
import requests
from requests.exceptions import Timeout
import time
import json

## Write a class to handle the GET requests to NewYorker.com

In [14]:
class Scraper(object):
    """Scrape cartoons from The New Yorker random cartoon page"""

    # Create a persistent requests connection
    _session = requests.Session()
    _session.headers = {'application': 'PyToons',
                       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
    _SLEEP_MIN = 0.2  # Enforce minimum wait time between API calls (seconds)

    def __init__(self, timeout=5, sleep_time=0.5):
        """ Scraper Constructor

        :param timeout: time before quitting on response (seconds)
        :param sleep_time: time to wait between requests
        """

        self.api_root = "https://www.newyorker.com"
        self.timeout = timeout
        self.sleep_time = sleep_time

    def _make_request(self, path, method='GET'):
        """Make a request to the API"""
        uri = self.api_root + path
        
        # Make the request
        response = None
        try:
            response = self._session.request(method, uri, timeout=self.timeout)
        except Timeout as e:
            print("Timeout raised and caught:\n{e}".format(e=e))

        # Enforce rate limiting
        time.sleep(max(self._SLEEP_MIN, self.sleep_time))
        return response

    def get_random_cartoon(self):
        """Requests a random New Yorker cartoon"""
        endpoint = "/cartoons/random/randomAPI"
        return self._make_request(endpoint)

## Test out the scraper

In [15]:
# Create an instance of the cartoon scraper
scraper = Scraper()

In [37]:
# Get a random cartoon
r = scraper.get_random_cartoon()

In [41]:
data = r.json()
data

[{'id': '1556571',
  'pn': 'a18169',
  'date': '07/14/2014 00:24',
  'src': 'https://www.newyorker.com/wp-content/uploads/2014/04/140421_a18169-600.jpg',
  'caption': '',
  'cartoon_bank_url': ''},
 {'id': '2861305',
  'pn': 'kaplan-2004-09-27',
  'date': '10/10/2014 17:58',
  'src': 'https://www.newyorker.com/wp-content/uploads/2014/10/141020_kaplan-2004-09-27-600.jpg',
  'caption': '&ldquo;I knew Mary would dump me when my fleece got a little gray.&rdquo;',
  'cartoon_bank_url': ''}]